# calculate rotation and transposition matrix

In [1]:
import os

old_folder = r'D:\Pu\20211229-P_brain_M1_nonclear\Alignment'

before_position_file = os.path.join(old_folder, '10x_positions_before.txt')

new_folder = r'D:\Pu\20220102-CTP11-1000_CTP12-DNA_from_1229\Alignment'

after_position_file = os.path.join(new_folder, '10x_positions_after.txt')

In [2]:
os.path.isfile(before_position_file), os.path.isfile(after_position_file)

(True, True)

In [3]:
import numpy as np
import os, sys
# 1. alignment for manually picked points
def align_manual_points(pos_file_before, pos_file_after,
                        save=True, save_folder=None, save_filename='', verbose=True):
    """Function to align two manually picked position files, 
    they should follow exactly the same order and of same length.
    Inputs:
        pos_file_before: full filename for positions file before translation
        pos_file_after: full filename for positions file after translation
        save: whether save rotation and translation info, bool (default: True)
        save_folder: where to save rotation and translation info, None or string (default: same folder as pos_file_before)
        save_filename: filename specified to save rotation and translation points
        verbose: say something! bool (default: True)
    Outputs:
        R: rotation for positions, 2x2 array
        T: traslation of positions, array of 2
    Here's example for how to translate points
        translated_ps_before = np.dot(ps_before, R) + t
    """
    # load position_before
    if os.path.isfile(pos_file_before):
        ps_before = np.loadtxt(pos_file_before, delimiter=',')

    # load position_after
    if os.path.isfile(pos_file_after):
        ps_after = np.loadtxt(pos_file_after, delimiter=',')

    # do SVD decomposition to get best fit for rigid-translation
    c_before = np.mean(ps_before, axis=0)
    c_after = np.mean(ps_after, axis=0)
    H = np.dot((ps_before - c_before).T, (ps_after - c_after))
    U, _, V = np.linalg.svd(H)  # do SVD
    # calcluate rotation
    R = np.dot(V, U.T).T
    if np.linalg.det(R) < 0:
        R[:, -1] = -1 * R[:, -1]
    # calculate translation
    t = - np.dot(c_before, R) + c_after
    # here's example for how to translate points
    # translated_ps_before = np.dot(ps_before, R) + t
    # save
    if save:
        if save_folder is None:
            save_folder = os.path.dirname(pos_file_before)
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)
        if len(save_filename) > 0:
            save_filename += '_'
        rotation_name = os.path.join(save_folder, save_filename+'rotation')
        translation_name = os.path.join(
            save_folder, save_filename+'translation')
        np.save(rotation_name, R)
        np.save(translation_name, t)

    return R, t

In [4]:
R, T = align_manual_points(before_position_file, after_position_file, save=False, save_folder=new_folder)


In [5]:
R, T

(array([[ 0.99990725, -0.0136195 ],
        [ 0.0136195 ,  0.99990725]]),
 array([2817.49498772, -216.30549335]))

# transpose 60x positions

In [17]:
old_positions = np.loadtxt(os.path.join(os.path.dirname(old_folder), 'positions_all.txt'), delimiter=',')

In [18]:
new_positions = np.dot(old_positions, R) + T
print(new_positions)

[[ -913.53251088  -839.04853402]
 [ -916.25641032 -1039.02998409]
 [ -918.98030975 -1239.01143416]
 [ -921.70420919 -1438.99288423]
 [ -724.44665856 -1641.69823373]
 [ -721.72275912 -1441.71678366]
 [ -718.99885969 -1241.73533359]
 [ -716.27496025 -1041.75388353]
 [ -713.55106081  -841.77243346]
 [ -513.56961074  -844.49633289]
 [ -516.29351018 -1044.47778296]
 [ -519.01740962 -1244.45923303]
 [ -521.74130905 -1444.4406831 ]
 [ -524.46520849 -1644.42213317]
 [ -324.48375842 -1647.14603261]
 [ -321.75985898 -1447.16458254]
 [ -319.03595955 -1247.18313247]
 [ -316.31206011 -1047.2016824 ]
 [ -313.58816067  -847.22023233]
 [ -110.88281116  -649.9626817 ]
 [ -113.6067106   -849.94413177]
 [ -116.33061004 -1049.92558184]
 [ -119.05450948 -1249.90703191]
 [ -121.77840891 -1449.88848198]
 [ -124.50230835 -1649.86993205]
 [ -127.22620779 -1849.85138212]
 [   72.75524228 -1852.57528155]
 [   75.47914172 -1652.59383148]
 [   78.20304115 -1452.61238142]
 [   80.92694059 -1252.63093135]
 [   83.65

In [8]:
save_filename = os.path.join(new_folder, 'translated_positions_all.txt')
print(save_filename)
np.savetxt(save_filename, new_positions, fmt='%.2f', delimiter=',')

D:\Pu\20220102-CTP11-1000_CTP12-DNA_from_1229\Alignment\translated_positions_all.txt


In [10]:
# translate fine adjust positions if applicable
old_fine_positions = np.loadtxt(os.path.join(os.path.dirname(old_folder), 'positions_fine_adjust_before.txt'), delimiter=',')
new_fine_positions = np.dot(old_fine_positions, R) + T
print(new_fine_positions) 


[[-1257.14813779 -1492.79929711]
 [-1427.50624004  1416.83076176]
 [ 6654.32700491 -1297.39148657]
 [ 6114.52176835  1858.73311879]]


# further adjust manually

In [15]:
#before_adjust = np.array([[-6101.45,-2075.75],
#                         [-4498.45,-2250.06],
#                         [-5118.13,100.40],
#                         ])
before_adjust = new_fine_positions[:3]


after_adjust = np.array([[-1259., -1473.8],
                        [-1430.0, 1436.0],
                        [6653.,-1278.5],
                        ])


after_adjust -before_adjust

array([[-1.85186221, 18.99929711],
       [-2.49375996, 19.16923824],
       [-1.32700491, 18.89148657]])

In [19]:
manual_shift = np.mean(after_adjust -before_adjust, axis=0)
manual_shift

array([-1.8908757 , 19.02000731])

In [20]:
#manual_shift = np.array([-28.1,  -8.7])
adjusted_new_positions = new_positions + manual_shift

In [21]:
adj_save_filename = os.path.join(new_folder, 'adjusted_translated_positions_all.txt')
print(adj_save_filename)
np.savetxt(adj_save_filename, adjusted_new_positions, fmt='%.2f', delimiter=',')

D:\Pu\20220102-CTP11-1000_CTP12-DNA_from_1229\Alignment\adjusted_translated_positions_all.txt


In [22]:

print(new_fine_positions + manual_shift)

[[-1259.03901348 -1473.7792898 ]
 [-1429.39711573  1435.85076907]
 [ 6652.43612922 -1278.37147927]
 [ 6112.63089266  1877.7531261 ]]
